In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score, KFold, ShuffleSplit,cross_val_predict
from sklearn import metrics

def from_features_to_classif_scores(features,labels,ftest,ltest):
    #stack them for learning
    features = features.reshape((len(features),-1))
    ftest = ftest.reshape((len(ftest),-1))
    # apply pipeline
    n = len(features)
    
    gs = GridSearchCV(SVC(kernel='rbf'), param_grad={'gamma',[0.01,0.05,0.1,0.11,0.15],'C',[0.8,0.9,1,1.1,1.2]})
    pipeline = make_pipeline(Normalizer(),StandardScaler(),gs)
    
    # pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
    cv = ShuffleSplit(n,n_iter=3,test_size=1, train_size=1)
    
    scores = cross_val_score(pipeline,features,labels,cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))

    return pipeline,scores

In [6]:
num_images_ta

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [69]:
n = 60000
from load_mnist_db import load_images_mnist
from sklearn.preprocessing import OneHotEncoder
import time as time
import numpy as np
from keras.datasets import mnist
num_images = n
px=32

Xtrain,ytrain,Xtest,ytest = load_images_mnist(px=px)

Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
Xtest_1d = Xtest.reshape((len(Xtest),-1))

num_samples,num_features = Xtrain_1d.shape


#num2vec=OneHotEncoder()

#ytrain_1d.shape=(num_samples,1)
#num2vec.fit(ytrain_1d)

#ytrain=num2vec.transform(ytrain_1d)

#ytest_1d.shape=(ytest_1d.shape[0],1)
#ytest=num2vec.transform(ytest_1d)


X_train shape: (60000, 32, 32)
60000 train samples
10000 test samples


In [67]:
ytest.shape

(10000, 10)

In [68]:
################## Linear SVM

from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

ns = [300, 1000, 2000, 5000, 10000]
score = np.zeros((5,1))
for i,n in enumerate(ns):
    pip_linear = make_pipeline(SVC(kernel='linear',C=1))

    pip_linear.fit(Xtrain_1d[0:n,:],ytrain[0:n,:])
    out=pip_linear.predict(Xtest_1d)
    score[i] = 1.0-accuracy_score(ytest, out)

score

ValueError: bad input shape (300, 10)

In [49]:
ytrain[0:3].transpose()

array([5, 0, 4], dtype=uint8)

In [62]:
from sklearn.preprocessing import OneHotEncoder
vv=OneHotEncoder()
vector =ytrain[0:10]
print(vector)

vector.shape=(10,1)
aa=vv.fit(vector)
output=vv.transform(vector[0:4,:])
output.toarray()

[5 0 4 1 9 2 1 3 1 4]


array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [88]:
Cvec = np.arange(0.01,2,0.2)
Cvec.shape
Xtrain_1d.shape

(60000, 1024)

In [84]:
################## Cross-validation for Gaussian SVM
gammavec = np.array(10**np.arange(-3,1,0.4))
gammavec.shape = (10)
Cvec = np.arange(0.01,2,0.2)
Cvec.shape=(10)
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf'),parameters)
pip_gaussian = make_pipeline(Normalizer(),gs_gaussian)
n = 4000
pip_gaussian.fit(Xtrain_1d[0:n,:],ytrain[0:n].ravel())

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(bestgamma)
print(bestC)

1.58489319246
1.41


In [86]:
gs_gaussian.best_score_
bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(bestgamma)
print(bestC)

1.58489319246
1.41


In [105]:
a=np.histogram(ytrain[0:4000])


array([ 0.09475,  0.111  ,  0.09925,  0.098  ,  0.1075 ,  0.0875 ,
        0.10125,  0.10825,  0.09325,  0.09925])

In [ ]:
def sort_scats_per_class(X,y)

    num_classes = y.max()-y.min()
    


Xtrain_well_distributed

In [87]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, Normalizer

num_items,num_features = Xtrain_1d.shape
#parameters = {"C":[1],
#              "gamma":[1./num_features]}

parameters = {"C":[np.arange(0.01,2,10)],
              "gamma":[10**np.arange(-3,1,10)]}


ns = [300, 1000, 2000, 5000, 10000]
score_gaussian = np.zeros((5,1))
for i,n in enumerate(ns):
    #  gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)
    gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
    pip_gaussian = make_pipeline(Normalizer(),gs_gaussian)

    pip_gaussian.fit(Xtrain_1d[0:n,:],ytrain[0:n])
    out=pip_gaussian.predict(Xtest_1d)
    score_gaussian[i] = 1.0-accuracy_score(ytest, out)
    print(score_gaussian[i])
    
score_gaussian



[ 0.1533]
[ 0.0773]
[ 0.0594]
[ 0.0404]
[ 0.0327]


array([[ 0.1533],
       [ 0.0773],
       [ 0.0594],
       [ 0.0404],
       [ 0.0327]])

In [36]:
Xtrain_1d.shape

(60000, 1024)

In [99]:
J=3
L=6
N=32
uu=4#N/2**(J)

4096/(J*L+J*(J-1)*L*L/2)
#(1+J*L+J*(J-1)*L*L/2)*uu*uu


46.0

In [36]:
print(gamma)
parameters['gamma']


0.0004921259842519685


[4.9212598425196856e-05, 0.0004921259842519685, 0.004921259842519685]

In [35]:
pip_gaussian2 = make_pipeline(StandardScaler(),SVC(kernel='rbf',C=C,gamma=gamma))

pip_gaussian2.fit(Xtrain_1d[0:n,:],ytrain[0:n].ravel())
out=pip_gaussian2.predict(Xtest_1d)
score = accuracy_score(ytest, out)
score

#for n in np.arange(300
#X = normalize(Xtrain_1d[2*n:3*n,:])
#X = scale(X)

#gs_linear.fit(X,ytrain[2*n:3*n])
#gs_linear.best_params_

#Xtest_1d = Xtest.reshape((len(Xtest),-1))
#Xe = normalize(Xtest_1d)
#Xe = scale(Xe)

#predictedY = gs_linear.predict(Xtest_1d)
#score = accuracy_score(ytest, predictedY)
#score



0.84819999999999995

In [ ]:
gs_linear.best_params_['C']

In [ ]:
Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xe = normalize(Xtest_1d)
Xe = scale(Xe)

predictedY = gs_linear.predict(Xe)
score = accuracy_score(ytest, predictedY)
score

In [ ]:

parameters = {"C":[1,1.1,1.11,1.05] ,
              "gamma":[0.1/num_features,1/num_features,10./num_features]}
gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)

X = normalize(Xtrain_1d[0:n,:])
X = scale(X)

gs_gaussian.fit(X,ytrain[0:n])
gs_gaussian.best_params_

Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xe = normalize(Xtest_1d)
Xe = scale(Xe)

predictedY = gs_gaussian.predict(Xe)
score = accuracy_score(ytest, predictedY)
score


In [ ]:
from sklearn.metrics import accuracy_score
print('mine:',np.sum(ytest==predictedY)/len(Xtest))
print('two:',accuracy_score(ytest, predictedY))

In [ ]:

#sampling the whole dataset 
n = 300
train_size=n/60000
test_size=0.5*n/60000

pipeline = make_pipeline(Normalizer(),StandardScaler(),gs)

# pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
cv = ShuffleSplit(n,n_iter=3,test_size=test_size, train_size=train_size)

scores = cross_val_score(pipeline,Xtrain_1d[0:n,:],ytrain[0:n],cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))


In [ ]:
gs.best_params_
pipeline.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score, KFold, ShuffleSplit,cross_val_predict
from sklearn import metrics

def from_features_to_classif_scores(features,labels,ftest,ltest):
    #stack them for learning
    features = features.reshape((len(features),-1))
    ftest = ftest.reshape((len(ftest),-1))
    # apply pipeline
    n = len(features)
    
    pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
    cv = ShuffleSplit(n,n_iter=3,test_size=0.1, train_size=0.9)
    
    scores = cross_val_score(pipeline,features,labels,cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))
    print('min score:',(1.0-scores.max())*100)
    return pipeline,scores

In [ ]:
from load_mnist_db import load_scattering_mnist
import time as time
import numpy as np
from keras.datasets import mnist

num_images = 60000
px=32

Xtrain,ytrain,Xtest,ytest = load_scattering_mnist(num_images = num_images, px=px,J=3,L=8)

num_points = [330, 1100, 2200, 5500, 11000, 22000, 60000]
for n in num_points:
    print('n=',n)
    pipeline, test_err = from_features_to_classif_scores(Xtrain[0:n,:,:,:],ytrain[0:n],Xtest[0:n,:,:,:],ytest[0:n])


In [ ]:
test_err

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.svm import SVC

X_train = Xtrain
y_train = ytrain
X_train = features.reshape((len(X_train),-1))
X_test = Xtest.reshape((len(Xtest),-1))

# apply pipeline
n = len(X_train)

pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=True))
#cv = ShuffleSplit(n,n_iter=3,test_size=1, train_size=1)

normalizer = Normalizer()
scaler = StandardScaler().fit(X_train)
X_train_normalized = normalizer.transform(X_train)
X_train_transformed = scaler.transform(X_train_normalized)

clf = SVC(kernel='rbf', C=1, verbose=True).fit(X_train_transformed, y_train)

X_test_normalized = normalizer.transform(X_test)
X_test_transformed = scaler.transform(X_test_normalized)

clf.score(Xtest, ytest)   


print('score:',scores)